## Introduction to Linear Regression

In this notebook, we'll see how we can use the _statsmodels_ library to perform linear regression in Python.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

We'll be using a dataset on possums in Australia and New Guinea, borrowed from the OpenIntro Statistics textbook.

You can read more about it here: https://www.openintro.org/data/index.php?data=possum

Our goal will be to understand the relationship between the total length in cm (`total_l`) and the other variables in our dataset.

In [ ]:
possum = pd.read_csv('../data/possum.csv')

In [ ]:
possum.head()

In [ ]:
plt.hist(possum['total_l'], bins = 'fd', edgecolor = 'black');

Let's start by looking at the total length (`total_l`) vs. the length of the head in mm (`head_l`).

In [ ]:
possum.plot(
    kind = 'scatter',
    x = 'head_l',
    y = 'total_l'
);

**Question:** How would you describe this relationship?





Recall that the model we will be fitting is

$$(\text{total_l})_i = \beta_0 + \beta_1\cdot(\text{head_l})_i + \epsilon_i$$

where the $\epsilon_i$ values are independent and normally distributed with a mean of 0 and constant (across $i$) variance $\sigma^2$.

To fit our model, we'll use the [statsmodels](https://www.statsmodels.org/stable/index.html) library. Specifically, we'll be using the formula api.

In [ ]:
import statsmodels.formula.api as sm

We'll use the [ols class](https://www.statsmodels.org/devel/generated/statsmodels.formula.api.ols.html). 

For this, we need to specify the following:
* **formula:** A formula specifying the model. These use the [patsy formulas](https://patsy.readthedocs.io/en/latest/formulas.html), which look like "target ~ predictors".
* **data:** The dataset, as a _pandas_ DataFrame.

In [ ]:
lr = sm.ols(
    formula = 'total_l ~ head_l',
    data = possum
).fit()                                                  # We can go ahead and tell it to fit the model

Let's look at the output.

In [ ]:
lr.summary()

Filling in the coefficients from the model, we get

$$(\text{total_l})_i =  9.8882 + 0.8337\cdot(\text{head_l})_i + \epsilon_i$$

This says that for every one mm increase in the length of the head, the average total length increases by 0.8337 cm.

Let's overlay the line over our scatterplot.

To do this, we'll make use of the `predict` method of our fit model.

In [ ]:
x_pred = pd.DataFrame({'head_l': np.linspace(start = possum['head_l'].min(),
                                    stop = possum['head_l'].max(), num = 250)})

pred = lr.predict(x_pred)

possum.plot(
    kind = 'scatter',
    x = 'head_l',
    y = 'total_l',
    figsize = (10,6)
)

plt.plot(x_pred['head_l'], pred, color = 'black');

How good is the fit of this model? Let's look at the $R^2$ value. It is accessible through the `rsquared` attribute.

In [ ]:
lr.rsquared

Let's verify that this is the correct value. Recall the formula for $R^2$:

$$R^2 = \frac{TSS - RSS}{TSS}$$

TSS: Total sum of squares

RSS: Residual sum of squares

To compute TSS, we need to look at the difference between the target values and the average value of the target variable.

In [ ]:
tss = ((possum['total_l'] - possum['total_l'].mean())**2).sum()

For RSS, we need to consider the difference between the target and the predicted value.

In [ ]:
rss = ((possum['total_l'] - lr.fittedvalues)**2).sum()

Now, we can verify that we get the same result.

In [ ]:
(tss - rss) / tss

Let's look more closely at the results. First, is the slope coefficient statistically significant?

We could perform a hypothesis test with null hypothesis

$H_0: \beta_1 = 0$

and alternative hypothesis

$H_A: \beta_1 \neq 0$.

The result of this hypothesis test is present in the model summary. Specifically, it is the P>|t| value associated with head_l. We can access it through the `pvalues` attribute. 

In [ ]:
lr.pvalues['head_l']

Based on this, it is safe to reject the null hypothesis and conclude that there is a linear relationship between head length and total length.

We could have also looked at the confidence interval for $\beta_1$, which is also displayed in the summary, or we could access it through the `conf_int` method. To use this, you pass in 1 - confidence level.

In [ ]:
lr.conf_int(0.05)

In [ ]:
possum['head_l'].describe()

Now, let's use our model to make some predictions.

**Question:** For possum's with head length of 91, what average total length does the model predict? 

We can answer this using the `predict` method. It expects a DataFrame with `head_l` column.

In [ ]:
lr.predict(pd.DataFrame({'head_l': [91]}))

If we want a confidence interval for this estimated average, we need to use the `get_prediction` method and ask for the confidence interval.

In [ ]:
lr.get_prediction(pd.DataFrame({'head_l': [91]})).conf_int()

If we want a **prediction interval**, which tells us what we can expect for a new observation, we can specify `obs = True`.

In [ ]:
lr.get_prediction(pd.DataFrame({'head_l': [91]})).conf_int(obs = True)

Notice that the confidence interval here is much wider.

We can also get all of these results using the `summary_frame` method.

In [ ]:
lr.get_prediction(pd.DataFrame({'head_l': [91]})).summary_frame()

Let's display the confidence bands over a range of head_l values.

In [ ]:
x_pred = pd.DataFrame({'head_l': np.linspace(start = possum['head_l'].min(),
                                    stop = possum['head_l'].max(), num = 250)})

pred = lr.get_prediction(x_pred).summary_frame()

possum.plot(
    kind = 'scatter',
    x = 'head_l',
    y = 'total_l',
    figsize = (10,6)
)

plt.plot(x_pred['head_l'], pred['mean'], color = 'grey', label = 'predicted mean')

plt.plot(x_pred['head_l'], pred['mean_ci_lower'], color = 'blue', label = 'confidence interval')
plt.plot(x_pred['head_l'], pred['mean_ci_upper'], color = 'blue')

plt.plot(x_pred['head_l'], pred['obs_ci_lower'], color = 'black', label = 'prediction interval')
plt.plot(x_pred['head_l'], pred['obs_ci_upper'], color = 'black')

plt.legend();

A couple of things to notice:

* The prediction interval is much wider than the confidence interval for the mean.
* The width of the intervals are not constant. The further we are from the average value of `head_l`, the wider the interval grows.

## Diagnostics

In order to rely on these intervals, the assumptions of the linear model must be true. Remember the assumptions of the linear regression model:

1. The mean of the response at each value of the predictor is a linear function of the predictor variables.
2. The errors are independent.
3. The errors at each value of the predictor variables are normally distributed.
4. The errors have equal variance across all values of the predictor variables.

There are various consequences if these assumptions are violated. If the goal of building the model is to make accurate predictions (for example, when doing linear regression in the context of machine learning), but if our goal is inference, it is important to check these assumptions.


Based on the plots above, the first assumption that the mean response is a linear function of the predictor variables looks reasonable.

Now, let's take a look at the residuals to check the other three assumptions. A common way to do this is to look at the residuals vs. the fitted values.

In [ ]:
plt.scatter(x = lr.fittedvalues, y = lr.resid);

There don't appear to be any patterns in the residual plot. Let's check a histogram and QQ-plot.

In [ ]:
plt.hist(lr.resid, bins = 'fd', edgecolor = 'black');

In [ ]:
from scipy.stats import probplot

In [ ]:
probplot(lr.resid, plot = plt);

If we want to be extra sure, we can use a [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test). This test has the null hypothesis that the data comes from a normal distribution. 

In [ ]:
import statsmodels.api as stats

In [ ]:
stats.stats.diagnostic.kstest_normal(lr.resid)

The second value is the p-value. In this case, since the p-value is large, we don't have enough evidence to reject the null hypothesis. It looks like a normal distribution is a reasonable model for the residuals.

We also need that the residuals have constant variance. We can test that using a [White Test](https://en.wikipedia.org/wiki/White_test) for homoskedasticity. Here, the null hypothesis is that the variance of the residuals does not depend on the value of the predictors.

In [ ]:
stats.stats.het_white(lr.resid, lr.model.exog)

Here, we need to pay attention to the second output value, which is the p-value for the test. Again, the p-value is large, so there is not enough evidence to conclude that the variance depends on the value of the predictors.

## Categorical Predictors

Let's investigate whether the sex of the possum makes a statistically significant differene on the average total length.

**Null Hypothesis:** The average value of the total length does not depend linearly on the sex of the possum ($\beta_1 = 0$).

**Alternative Hypothesis:** The average value of the total length does depend linearly on the sex of the possum ($\beta_1 \neq 0$).

Before fitting our model, let's look at a boxplot.

In [ ]:
sns.boxplot(data = possum,
           x = 'sex', y = 'total_l');

In [ ]:
possum.groupby('sex')['total_l'].describe()

There are differences in the average values, but are these differences statistically significant?

Notice that we are using a categorical predictor variable in this case. Statsmodels allows for fitting models using categorical predictors in the same way as we used numeric predictors.

In [ ]:
lr_sex = sm.ols('total_l ~ sex', data = possum).fit()
lr_sex.summary()

**Question:** Looking at the summary output, what conclusion can we reach about the sex variable?

In [ ]:
cars = pd.read_csv('../data/auto-mpg.csv')

In [ ]:
cars.head(2)

In [ ]:
cars.plot(kind = 'scatter', x = 'displacement', y = 'mpg', figsize = (10,6));

In [ ]:
lr_cars = sm.ols('mpg ~ displacement', data = cars).fit()
lr_cars.summary()

If we plot the residuals against the values of the predictor, we can see a pattern. This is indicative that the assumption of linearity may not be correct and a different model or some kind of transformation might be appropriate.

In [ ]:
plt.figure(figsize = (10,6))
plt.scatter(cars['displacement'], lr_cars.resid)
xmin, xmax = plt.xlim()
plt.hlines(y = 0, xmin = xmin, xmax = xmax)
plt.xlim(xmin, xmax);

In [ ]:
probplot(lr_cars.resid, plot = plt);

In [ ]:
stats.stats.diagnostic.kstest_normal(lr_cars.resid)

In [ ]:
stats.stats.het_white(lr_cars.resid, lr_cars.model.exog)

Let's see how our confidence and prediction intervals look inaccurate when the assumptions are not met.

In [ ]:
var = 'displacement'

x_pred = pd.DataFrame({
    var: np.linspace(start = cars[var].min(),
                               stop = cars[var].max(), num = 250)
})

pred = lr_cars.get_prediction(x_pred).summary_frame()

cars.plot(kind = 'scatter', x = var, y = 'mpg', figsize = (10,6))

plt.plot(x_pred[var], pred['mean'], color = 'grey', label = 'predicted mean')

plt.plot(x_pred[var], pred['mean_ci_lower'], color = 'blue', label = 'confidence interval')
plt.plot(x_pred[var], pred['mean_ci_upper'], color = 'blue')

plt.plot(x_pred[var], pred['obs_ci_lower'], color = 'black', label = 'prediction interval')
plt.plot(x_pred[var], pred['obs_ci_upper'], color = 'black')

plt.legend();